<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/carbon-structures/Copy_of_Mercury_Graphene_Sheet_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and compile LAMMPS


In [ ]:
# Install dependencies and compile LAMMPS
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git
# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [37]:
!pip install MDAnalysis

import numpy as np
import MDAnalysis as mda
from MDAnalysis.lib.util import openany
from MDAnalysis.analysis.rdf import InterRDF
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import math, sys, os

np.random.seed(42)

In [42]:
!wget -O CH.airebo https://raw.githubusercontent.com/simongravelle/lammps-input-files/main/inputs/amorphous-carbon/CH.airebo
# Download necessary files
!wget -O PARM.lammps https://raw.githubusercontent.com/simongravelle/lammps-input-files/0e14b4b20a3ba8982e4b32c3a3574ecfe2f8f266/inputs/water-in-graphene-slit/PARM.lammps
!wget -O data.lammps https://raw.githubusercontent.com/simongravelle/lammps-input-files/0e14b4b20a3ba8982e4b32c3a3574ecfe2f8f266/inputs/water-in-graphene-slit/data.lammps


--2025-01-13 00:04:42--  https://raw.githubusercontent.com/simongravelle/lammps-input-files/main/inputs/amorphous-carbon/CH.airebo
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 926693 (905K) [text/plain]
Saving to: ‘CH.airebo’

CH.airebo           100%[===================>] 904.97K  --.-KB/s    in 0.04s   

2025-01-13 00:04:42 (21.7 MB/s) - ‘CH.airebo’ saved [926693/926693]

--2025-01-13 00:04:42--  https://raw.githubusercontent.com/simongravelle/lammps-input-files/0e14b4b20a3ba8982e4b32c3a3574ecfe2f8f266/inputs/water-in-graphene-slit/PARM.lammps
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443...

In [47]:
# Save the activated carbon script
with open("activated_carbon.in", "w") as f:
    f.write("""
# LAMMPS input file for water in 3-graphene slit
# LAMMPS input file for water in 3-graphene slit
variable    tem equal 300
variable    frc equal 2e-4  # Force applied to water molecules

units       real
atom_style  full
bond_style  harmonic
angle_style harmonic
dihedral_style opls
improper_style harmonic
boundary    p p p
pair_style  lj/cut/tip4p/long 1 2 1 1 0.105 10.0
kspace_style pppm/tip4p 1.0e-4
pair_modify mix arithmetic tail yes
special_bonds lj/coul 0.0 0.0 0.5 angle yes

read_data   data.lammps  # Load atomistic structure
include     PARM.lammps  # Load parameters

group       wat type 1 2  # Define water molecule group
group       Car type 3    # Define graphene group

fix         myshk wat shake 1.0e-4 200 0 b 1 a 1
fix         mynve all nve

compute     ctN wat temp/partial 0 1 1
fix         myber1 wat temp/berendsen ${tem} ${tem} 100
fix_modify  myber1 temp ctN

compute     ctM Car temp
fix         myber2 Car temp/berendsen ${tem} ${tem} 100
fix_modify  myber2 temp ctM

fix         kick wat addforce ${frc} 0.0 0.0

fix         mymum Car momentum 1 linear 1 1 1
fix         myrec Car recenter NULL NULL INIT

dump        dp1 all atom 100 dump.lammpstrj  # Output simulation data

timestep    2.0
thermo      100
run         1000

    """)

In [48]:
# Run the simulation
! /content/lammps/build/lmp -in activated_carbon.in

LAMMPS (29 Aug 2024 - Update 1)
  using 4 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (-40.834065 -40.734952 -16.552289) to (40.834065 40.734952 16.552289)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  22728 atoms
  reading velocities ...
  22728 velocities
  scanning bonds ...
  2 = max bonds/atom
  scanning angles ...
  3 = max angles/atom
  scanning dihedrals ...
  8 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  orthogonal box = (-40.834065 -40.734952 -16.552289) to (40.834065 40.734952 16.552289)
  1 by 1 by 1 MPI processor grid
  reading bonds ...
  27692 bonds
  reading angles ...
  47704 angles
  reading dihedrals ...
  90288 dihedrals
  reading impropers ...
  15150 impropers
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0.5     
  special bond factors coul:  0        0        0.5     
     3 = max # of 1-2 neighbors
     6 = max # of 1-3 neighbors
    18 = max # of 1-4 neighbors
